In [1]:
import pyteomics.mzml
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from datetime import datetime
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects import pandas2ri

In [2]:
def get_file_path(file_name):
    if (file_name == 'Ex_Auto_J3_30umTB_2ngQC_60m_1') | (file_name == 'Ex_Auto_J3_30umTB_2ngQC_60m_2') | (file_name == 'Ex_Auto_K13_30umTA_2ngQC_60m_1') |  (file_name == 'Ex_Auto_K13_30umTA_2ngQC_60m_2') |(file_name == 'Ex_Auto_W17_30umTB_2ngQC_60m_1') | (file_name == 'Ex_Auto_W17_30umTB_2ngQC_60m_2'): 
        file_path = 'data/2ng_Files/mzMLs/'
        
    if (file_name == 'Ex_Auto_J3_30umTB_02ngQC_60m_1') | (file_name == 'Ex_Auto_J3_30umTB_02ngQC_60m_2') | (file_name == 'Ex_Auto_K13_30umTA_02ngQC_60m_1') | (file_name == 'Ex_Auto_K13_30umTA_02ngQC_60m_2') | (file_name == 'Ex_Auto_W17_30umTA_02ngQC_60m_3') | (file_name == 'Ex_Auto_W17_30umTA_02ngQC_60m_4'): 
        file_path = 'data/02ng_Files/mzMLs/'
        
    if (file_name == 'OR11_20160122_PG_HeLa_CVB3_CT_A') | (file_name == 'OR11_20160122_PG_HeLa_CVB3_CT_B') | (file_name == 'OR11_20160122_PG_HeLa_CVB3_CT_C'):
            file_path = 'data/Project_PXD011163/mzMLs/'
    
    return(file_path)

In [3]:
def remove_precursor_peak(df):
    #if there is a peak that is 10X more intense than everything else, get rid of it (it's a precursor that hasn't been fragmented)
    maxpeak = df['intensity'].max()
    
    #if it's ten times more intesne that all the other peaks get rid of it. 
    rm_max_peak = df[df['intensity']!=maxpeak]
    mean_intensity = rm_max_peak['intensity'].mean()
    
    if maxpeak >= 5*mean_intensity:
        
        #also remove the plusone isotope
        maxmz = df[df['intensity'] == maxpeak]['mz'].values[0]
        plusone = maxmz + 1.0034
        plusone_tol_max = plusone+.01
        plusone_tol_min = plusone-.01
        
        plusonemz_array = rm_max_peak[(rm_max_peak['mz'] <= plusone_tol_max) & (rm_max_peak['mz'] >= plusone_tol_min)]['mz'].values
        if len(plusonemz_array > 0 ): #if there is a plus1 isotope
            plusonemz = plusonemz_array[0]
            rm_max_peak = rm_max_peak[rm_max_peak['mz']!=plusonemz]
        
    return rm_max_peak

def get_dfs(scan1, scan2, mzml1, mzml2):
    spectrum_dict1 = mzml1.get_by_id("controllerType=0 controllerNumber=1 scan="+scan1)
    spectrum_id1 = spectrum_dict1['id']
    mz_array1 = spectrum_dict1['m/z array']
    intensity_array1 = spectrum_dict1['intensity array']
    
    spectrum_dict2 = mzml2.get_by_id("controllerType=0 controllerNumber=1 scan="+scan2)
    spectrum_id2 = spectrum_dict2['id']
    mz_array2 = spectrum_dict2['m/z array']
    intensity_array2 = spectrum_dict2['intensity array']
    
    
    df1 = pd.DataFrame({'mz':mz_array1, 'intensity':intensity_array1 })
    df2 = pd.DataFrame({'mz':mz_array2, 'intensity':intensity_array2 })

    df1 = remove_precursor_peak(df1)
    df2 = remove_precursor_peak(df2)
    
    return df1, df2

In [4]:
#get comparisons
internal = pd.read_csv('internal_combos.tsv', sep='\t')

In [5]:
internal['type'].unique()

array(['bulk good vs bulk good', '0.2ng good vs 0.2ng good'], dtype=object)

In [6]:
list(internal['File1'].unique())

['OR11_20160122_PG_HeLa_CVB3_CT_A',
 'OR11_20160122_PG_HeLa_CVB3_CT_B',
 'OR11_20160122_PG_HeLa_CVB3_CT_C',
 'Ex_Auto_K13_30umTA_02ngQC_60m_1',
 'Ex_Auto_J3_30umTB_02ngQC_60m_1',
 'Ex_Auto_J3_30umTB_02ngQC_60m_2',
 'Ex_Auto_K13_30umTA_02ngQC_60m_2',
 'Ex_Auto_W17_30umTA_02ngQC_60m_3',
 'Ex_Auto_W17_30umTA_02ngQC_60m_4']

In [7]:
list(internal['File2'].unique())

['OR11_20160122_PG_HeLa_CVB3_CT_A',
 'OR11_20160122_PG_HeLa_CVB3_CT_B',
 'OR11_20160122_PG_HeLa_CVB3_CT_C',
 'Ex_Auto_K13_30umTA_02ngQC_60m_1',
 'Ex_Auto_J3_30umTB_02ngQC_60m_2',
 'Ex_Auto_K13_30umTA_02ngQC_60m_2',
 'Ex_Auto_W17_30umTA_02ngQC_60m_4',
 'Ex_Auto_W17_30umTA_02ngQC_60m_3',
 'Ex_Auto_J3_30umTB_02ngQC_60m_1']

In [8]:
file1_list = ['Ex_Auto_J3_30umTB_02ngQC_60m_1','Ex_Auto_J3_30umTB_02ngQC_60m_2',
             'Ex_Auto_K13_30umTA_02ngQC_60m_1','Ex_Auto_K13_30umTA_02ngQC_60m_2',
             'Ex_Auto_W17_30umTA_02ngQC_60m_3','Ex_Auto_W17_30umTA_02ngQC_60m_4',]




In [9]:
# Defining the R script and loading the instance in Python
r = robjects.r
r['source']('calc_cosine_score.R')
SpectrumSimilarity = robjects.globalenv['SpectrumSimilarity']  # Loading the function we have defined in R.

In [10]:
sc02ngQ_v_sc02ngQ = []

In [11]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)  

counter_for_list_removal = 0
for file1_name in file1_list:
    print("File1: "+file1_name)
    #devide up scans based on file 

    rep1_rep1 = internal[(internal['File1']==file1_name)&
                                     (internal['File2']=='Ex_Auto_J3_30umTB_02ngQC_60m_1')]
    rep1_rep2 = internal[(internal['File1']==file1_name)&
                                     (internal['File2']=='Ex_Auto_J3_30umTB_02ngQC_60m_2')]
    rep1_rep3 = internal[(internal['File1']==file1_name)&
                                     (internal['File2']=='Ex_Auto_K13_30umTA_02ngQC_60m_1')]
    rep1_rep4 = internal[(internal['File1']==file1_name)&
                                     (internal['File2']=='Ex_Auto_K13_30umTA_02ngQC_60m_2')]
    rep1_rep5 = internal[(internal['File1']==file1_name)&
                                     (internal['File2']=='Ex_Auto_W17_30umTA_02ngQC_60m_3')]
    rep1_rep6 = internal[(internal['File1']==file1_name)&
                                     (internal['File2']=='Ex_Auto_W17_30umTA_02ngQC_60m_4')]


    rep1_rep1 = rep1_rep1[['Scan1','Scan2', 'type']].to_numpy()
    rep1_rep2 = rep1_rep2[['Scan1','Scan2', 'type']].to_numpy()
    rep1_rep3 = rep1_rep3[['Scan1','Scan2', 'type']].to_numpy()
    rep1_rep4 = rep1_rep4[['Scan1','Scan2', 'type']].to_numpy()
    rep1_rep5 = rep1_rep5[['Scan1','Scan2', 'type']].to_numpy()
    rep1_rep6 = rep1_rep6[['Scan1','Scan2', 'type']].to_numpy()
    
    
    file1_path = get_file_path(file1_name)
    file1_path = file1_path+file1_name+".mzML"
    file1 = pyteomics.mzml.MzML(file1_path)
    
    list_of_scans = [rep1_rep1,rep1_rep2,rep1_rep3,rep1_rep4,rep1_rep5,rep1_rep6]
    file2_list = ['Ex_Auto_J3_30umTB_02ngQC_60m_1','Ex_Auto_J3_30umTB_02ngQC_60m_2',
             'Ex_Auto_K13_30umTA_02ngQC_60m_1','Ex_Auto_K13_30umTA_02ngQC_60m_2',
             'Ex_Auto_W17_30umTA_02ngQC_60m_3','Ex_Auto_W17_30umTA_02ngQC_60m_4',]


    
    #remove items from list1 so that we don't make duplicate comparisons
    list_of_scans = list_of_scans[counter_for_list_removal: len(list_of_scans)]
    file2_list = file2_list[counter_for_list_removal: len(file2_list)]


    
    for i in range(0, len(file2_list)):
        file2_name = file2_list[i]
        file2_path = get_file_path(file2_name)
        file2_path = file2_path+file2_name+".mzML"
        file2 = pyteomics.mzml.MzML(file2_path)

        scans_to_compare = list_of_scans[i]

        print("File2: "+file2_name)
        for scan_pair in scans_to_compare:
            scan1 = scan_pair[0]
            scan2 = scan_pair[1]
            comp_typo = scan_pair[2]

            df1, df2 = get_dfs(str(scan1), str(scan2), file1, file2)

            pandas2ri.activate()
            mins = [df1['mz'].min(), df2['mz'].min()]
            maxs = [df1['mz'].max(), df2['mz'].max()]
            sim_score = SpectrumSimilarity(df1, df2, xlim = [min(mins), max(maxs)])
            sim_score = sim_score[0]
            
            sc02ngQ_v_sc02ngQ.append(sim_score)
    
    counter_for_list_removal += 1
               
                
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)  


Current Time = 22:08:48
File1: Ex_Auto_J3_30umTB_02ngQC_60m_1
File2: Ex_Auto_J3_30umTB_02ngQC_60m_1
File2: Ex_Auto_J3_30umTB_02ngQC_60m_2
File2: Ex_Auto_K13_30umTA_02ngQC_60m_1
File2: Ex_Auto_K13_30umTA_02ngQC_60m_2
File2: Ex_Auto_W17_30umTA_02ngQC_60m_3
File2: Ex_Auto_W17_30umTA_02ngQC_60m_4
File1: Ex_Auto_J3_30umTB_02ngQC_60m_2
File2: Ex_Auto_J3_30umTB_02ngQC_60m_2
File2: Ex_Auto_K13_30umTA_02ngQC_60m_1
File2: Ex_Auto_K13_30umTA_02ngQC_60m_2
File2: Ex_Auto_W17_30umTA_02ngQC_60m_3
File2: Ex_Auto_W17_30umTA_02ngQC_60m_4
File1: Ex_Auto_K13_30umTA_02ngQC_60m_1
File2: Ex_Auto_K13_30umTA_02ngQC_60m_1
File2: Ex_Auto_K13_30umTA_02ngQC_60m_2
File2: Ex_Auto_W17_30umTA_02ngQC_60m_3
File2: Ex_Auto_W17_30umTA_02ngQC_60m_4
File1: Ex_Auto_K13_30umTA_02ngQC_60m_2
File2: Ex_Auto_K13_30umTA_02ngQC_60m_2
File2: Ex_Auto_W17_30umTA_02ngQC_60m_3
File2: Ex_Auto_W17_30umTA_02ngQC_60m_4
File1: Ex_Auto_W17_30umTA_02ngQC_60m_3
File2: Ex_Auto_W17_30umTA_02ngQC_60m_3
File2: Ex_Auto_W17_30umTA_02ngQC_60m_4
File1:

In [12]:
with open('sc02ngQ_v_sc02ngQ.txt', 'w') as f1:
    for item in sc02ngQ_v_sc02ngQ:
        f1.write("%s\n" % item)

